In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
act, act_name = "relu", "relu"
print(act_name)

relu


In [4]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = activations[3], names[3]
print(act_name)


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

relu
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_____

In [3]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [6]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 148s - loss: 1.9521 - acc: 0.4123 - val_loss: 1.4238 - val_acc: 0.5036
Epoch 2/75
390/390 [==============================] - 143s - loss: 1.3625 - acc: 0.5678 - val_loss: 1.1092 - val_acc: 0.6505
Epoch 3/75
390/390 [==============================] - 143s - loss: 1.2461 - acc: 0.6244 - val_loss: 1.3889 - val_acc: 0.6417
Epoch 4/75
390/390 [==============================] - 142s - loss: 1.1942 - acc: 0.6501 - val_loss: 0.9129 - val_acc: 0.7195
Epoch 5/75
390/390 [==============================] - 142s - loss: 1.1308 - acc: 0.6699 - val_loss: 0.9396 - val_acc: 0.7028
Epoch 6/75
390/390 [==============================] - 143s - loss: 1.1085 - acc: 0.6817 - val_loss: 0.7849 - val_acc: 0.7551
Epoch 7/75
390/390 [==============================] - 143s - loss: 1.0347 - acc: 0.7017 - val_loss: 0.7559 - val_acc: 0.7657
Epoch 8/75
390/390 [==============================] - 143s - loss: 1.0564 - acc: 0.7066 - val_loss: 0.7671 - val_acc: 0.7661


390/390 [==============================] - 142s - loss: 0.6409 - acc: 0.8338 - val_loss: 0.5420 - val_acc: 0.8654
Epoch 67/75
390/390 [==============================] - 142s - loss: 0.6370 - acc: 0.8370 - val_loss: 0.5516 - val_acc: 0.8673
Epoch 68/75
390/390 [==============================] - 143s - loss: 0.6430 - acc: 0.8338 - val_loss: 0.5433 - val_acc: 0.8667
Epoch 69/75
390/390 [==============================] - 142s - loss: 0.6279 - acc: 0.8385 - val_loss: 0.5544 - val_acc: 0.8691
Epoch 70/75
390/390 [==============================] - 142s - loss: 0.6452 - acc: 0.8353 - val_loss: 0.5369 - val_acc: 0.8657
Epoch 71/75
390/390 [==============================] - 143s - loss: 0.6315 - acc: 0.8380 - val_loss: 0.5711 - val_acc: 0.8565
Epoch 72/75
390/390 [==============================] - 142s - loss: 0.6306 - acc: 0.8386 - val_loss: 0.5519 - val_acc: 0.8592
Epoch 73/75
390/390 [==============================] - 142s - loss: 0.6337 - acc: 0.8370 - val_loss: 0.5468 - val_acc: 0.8642
Epoc

In [7]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.5859 - acc: 0.8509 - val_loss: 0.4887 - val_acc: 0.8847
Epoch 2/25
390/390 [==============================] - 145s - loss: 0.5720 - acc: 0.8523 - val_loss: 0.4691 - val_acc: 0.8867
Epoch 3/25
390/390 [==============================] - 142s - loss: 0.5620 - acc: 0.8577 - val_loss: 0.4926 - val_acc: 0.8810
Epoch 4/25
390/390 [==============================] - 142s - loss: 0.5546 - acc: 0.8589 - val_loss: 0.4842 - val_acc: 0.8850
Epoch 5/25
390/390 [==============================] - 142s - loss: 0.5574 - acc: 0.8584 - val_loss: 0.4627 - val_acc: 0.8910
Epoch 6/25
390/390 [==============================] - 142s - loss: 0.5548 - acc: 0.8590 - val_loss: 0.4836 - val_acc: 0.8837
Epoch 7/25
390/390 [==============================] - 142s - loss: 0.5452 - acc: 0.8579 - val_loss: 0.4590 - val_acc: 0.8911
Epoch 8/25
390/390 [==============================] - 142s - loss: 0.5433 - acc: 0.8595 - val_loss: 0.4612 - val_acc: 0.8892


In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 143s - loss: 0.4817 - acc: 0.8753 - val_loss: 0.4213 - val_acc: 0.8949
Epoch 2/25
390/390 [==============================] - 142s - loss: 0.4748 - acc: 0.8758 - val_loss: 0.4332 - val_acc: 0.8945
Epoch 3/25
390/390 [==============================] - 143s - loss: 0.4674 - acc: 0.8776 - val_loss: 0.4215 - val_acc: 0.8944
Epoch 4/25
390/390 [==============================] - 154s - loss: 0.4725 - acc: 0.8770 - val_loss: 0.4211 - val_acc: 0.8950
Epoch 5/25
390/390 [==============================] - 148s - loss: 0.4647 - acc: 0.8782 - val_loss: 0.4157 - val_acc: 0.8954
Epoch 6/25
390/390 [==============================] - 143s - loss: 0.4689 - acc: 0.8768 - val_loss: 0.4240 - val_acc: 0.8955
Epoch 7/25
390/390 [==============================] - 143s - loss: 0.4647 - acc: 0.8786 - val_loss: 0.4261 - val_acc: 0.8942
Epoch 8/25
390/390 [==============================] - 142s - loss: 0.4605 - acc: 0.8774 - val_loss: 0.4337 - val_acc: 0.8908


In [9]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 90.120 loss: 0.402


In [4]:
baseMapNum = 32
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.load_weights("models/cifar10_normal_rms_ep125relu.h5")

In [5]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.00015,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep150'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 146s - loss: 0.4297 - acc: 0.8862 - val_loss: 0.3948 - val_acc: 0.9031
Epoch 2/25
390/390 [==============================] - 143s - loss: 0.4185 - acc: 0.8904 - val_loss: 0.3927 - val_acc: 0.9020
Epoch 3/25
390/390 [==============================] - 143s - loss: 0.4128 - acc: 0.8912 - val_loss: 0.3882 - val_acc: 0.9030
Epoch 4/25
390/390 [==============================] - 143s - loss: 0.4185 - acc: 0.8885 - val_loss: 0.3931 - val_acc: 0.9027
Epoch 5/25
390/390 [==============================] - 143s - loss: 0.4164 - acc: 0.8912 - val_loss: 0.3895 - val_acc: 0.9031
Epoch 6/25
390/390 [==============================] - 143s - loss: 0.4121 - acc: 0.8900 - val_loss: 0.3906 - val_acc: 0.9027
Epoch 7/25
390/390 [==============================] - 143s - loss: 0.4106 - acc: 0.8925 - val_loss: 0.3937 - val_acc: 0.9028
Epoch 8/25
390/390 [==============================] - 143s - loss: 0.4066 - acc: 0.8931 - val_loss: 0.3902 - val_acc: 0.9025


In [7]:
print(his.history)

{'val_acc': [0.90310000000000001, 0.90200000000000002, 0.90300000000000002, 0.90269999999999995, 0.90310000000000001, 0.90269999999999995, 0.90280000000000005, 0.90249999999999997, 0.90500000000000003, 0.90500000000000003, 0.90449999999999997, 0.90469999999999995, 0.9022, 0.90100000000000002, 0.90359999999999996, 0.90280000000000005, 0.90339999999999998, 0.90439999999999998, 0.90680000000000005, 0.9052, 0.9042, 0.90259999999999996, 0.90400000000000003, 0.90480000000000005, 0.90659999999999996], 'acc': [0.88621794871794868, 0.89033926848918532, 0.89118142448482818, 0.88847449466820361, 0.89122152709027758, 0.88995829319242559, 0.89250481231953804, 0.89304619826756493, 0.88853464873891863, 0.89268527428309119, 0.88825393009920095, 0.89029916584549096, 0.89310635226179014, 0.89170275906320184, 0.8923444016491483, 0.8925649662755184, 0.89320660889970982, 0.8938683028553096, 0.89605389799794821, 0.89579323067051653, 0.89402871348745439, 0.8937680461600227, 0.89483076674379058, 0.89362768685

In [8]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000075,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep175'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 144s - loss: 0.3835 - acc: 0.8977 - val_loss: 0.3778 - val_acc: 0.9064
Epoch 2/25
390/390 [==============================] - 143s - loss: 0.3854 - acc: 0.8969 - val_loss: 0.3731 - val_acc: 0.9087
Epoch 3/25
390/390 [==============================] - 143s - loss: 0.3776 - acc: 0.8985 - val_loss: 0.3724 - val_acc: 0.9079
Epoch 4/25
390/390 [==============================] - 143s - loss: 0.3797 - acc: 0.8977 - val_loss: 0.3732 - val_acc: 0.9087
Epoch 5/25
390/390 [==============================] - 143s - loss: 0.3753 - acc: 0.8998 - val_loss: 0.3745 - val_acc: 0.9081
Epoch 6/25
390/390 [==============================] - 143s - loss: 0.3741 - acc: 0.9004 - val_loss: 0.3737 - val_acc: 0.9072
Epoch 7/25
390/390 [==============================] - 143s - loss: 0.3769 - acc: 0.9004 - val_loss: 0.3710 - val_acc: 0.9071
Epoch 8/25
390/390 [==============================] - 143s - loss: 0.3798 - acc: 0.9002 - val_loss: 0.3718 - val_acc: 0.9095


In [9]:
print(his.history)

{'val_acc': [0.90639999999999998, 0.90869999999999995, 0.90790000000000004, 0.90869999999999995, 0.90810000000000002, 0.90720000000000001, 0.90710000000000002, 0.90949999999999998, 0.90920000000000001, 0.90690000000000004, 0.90790000000000004, 0.90859999999999996, 0.90790000000000004, 0.9083, 0.90800000000000003, 0.90659999999999996, 0.90769999999999995, 0.90949999999999998, 0.90849999999999997, 0.9073, 0.90800000000000003, 0.91000000000000003, 0.90920000000000001, 0.90669999999999995, 0.90880000000000005], 'acc': [0.8977163461538461, 0.89693615652255076, 0.89848010906012044, 0.8976981071351926, 0.89970324027603321, 0.90034488293846948, 0.900364934231633, 0.90018447222983489, 0.90048524220096093, 0.89904154637780065, 0.89948267566878559, 0.90200914342624472, 0.89992380492152557, 0.89860041702944837, 0.90060555024677869, 0.89950272701931644, 0.90056544758396195, 0.90064565287135068, 0.90237006741071846, 0.90156801407789244, 0.90082611489227116, 0.90094642286159921, 0.90088626881000666, 

In [10]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000035,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep180'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 144s - loss: 0.3635 - acc: 0.9026 - val_loss: 0.3669 - val_acc: 0.9107
Epoch 2/5
390/390 [==============================] - 143s - loss: 0.3648 - acc: 0.9013 - val_loss: 0.3656 - val_acc: 0.9105
Epoch 3/5
390/390 [==============================] - 143s - loss: 0.3617 - acc: 0.9034 - val_loss: 0.3671 - val_acc: 0.9106
Epoch 4/5
390/390 [==============================] - 143s - loss: 0.3628 - acc: 0.9040 - val_loss: 0.3641 - val_acc: 0.9111
Epoch 5/5
390/390 [==============================] - 144s - loss: 0.3654 - acc: 0.9026 - val_loss: 0.3646 - val_acc: 0.9104


In [11]:
print(his.history)

{'val_acc': [0.91069999999999995, 0.91049999999999998, 0.91059999999999997, 0.91110000000000002, 0.91039999999999999], 'acc': [0.90264423076923073, 0.9012672441067664, 0.90339268525517979, 0.9040142765862077, 0.90259063205621082], 'val_loss': [0.36693520574569705, 0.36561110734939573, 0.36710445284843446, 0.3641400626182556, 0.36457973694801332], 'loss': [0.36354731019490805, 0.36479970775508791, 0.3616125239157929, 0.36273352475284115, 0.36533849384488681]}


In [12]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000017,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep185'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 145s - loss: 0.3602 - acc: 0.9056 - val_loss: 0.3633 - val_acc: 0.9111
Epoch 2/5
390/390 [==============================] - 143s - loss: 0.3603 - acc: 0.9050 - val_loss: 0.3650 - val_acc: 0.9112
Epoch 3/5
390/390 [==============================] - 143s - loss: 0.3634 - acc: 0.9016 - val_loss: 0.3641 - val_acc: 0.9114
Epoch 4/5
390/390 [==============================] - 144s - loss: 0.3565 - acc: 0.9059 - val_loss: 0.3619 - val_acc: 0.9108
Epoch 5/5
390/390 [==============================] - 144s - loss: 0.3604 - acc: 0.9034 - val_loss: 0.3642 - val_acc: 0.9110


In [13]:
print(his.history)

{'val_acc': [0.91110000000000002, 0.91120000000000001, 0.91139999999999999, 0.91080000000000005, 0.91100000000000003], 'acc': [0.90556891025641029, 0.90499679176785219, 0.90160811675983166, 0.90595925565633328, 0.90337263392377132], 'val_loss': [0.36325090560913087, 0.3649802083492279, 0.36405324535369871, 0.36188883452415466, 0.36420677628517151], 'loss': [0.36020834465057422, 0.36033351676831998, 0.36339769352503531, 0.35645840020374031, 0.3604654489245826]}


In [14]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000012,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep190'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 145s - loss: 0.3582 - acc: 0.9034 - val_loss: 0.3641 - val_acc: 0.9115
Epoch 2/5
390/390 [==============================] - 143s - loss: 0.3558 - acc: 0.9050 - val_loss: 0.3639 - val_acc: 0.9100
Epoch 3/5
390/390 [==============================] - 143s - loss: 0.3587 - acc: 0.9053 - val_loss: 0.3626 - val_acc: 0.9122
Epoch 4/5
390/390 [==============================] - 142s - loss: 0.3611 - acc: 0.9030 - val_loss: 0.3633 - val_acc: 0.9122
Epoch 5/5
390/390 [==============================] - 143s - loss: 0.3574 - acc: 0.9057 - val_loss: 0.3625 - val_acc: 0.9120


In [15]:
print(his.history)

{'val_acc': [0.91149999999999998, 0.91000000000000003, 0.91220000000000001, 0.91220000000000001, 0.91200000000000003], 'acc': [0.90342548076923079, 0.90503689446891389, 0.90525745911440636, 0.90307186399089012, 0.90575874238049403], 'val_loss': [0.36410191831588745, 0.36391601028442383, 0.36261468706130984, 0.36325166683197019, 0.36246949920654298], 'loss': [0.35817433095131163, 0.35578855486837074, 0.35871736513600733, 0.3609324035294208, 0.35735944506255413]}
